# INSTRUÇÃO PRÁTICA 009
## Revisao e consolidaçao do conteúdo de __Pandas__ avançado.
Para a pratica serão utilizados dados de estações metereologicas de Salvador dos anos de 2003, 2013 e 2023, diponiveis em: [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos). 

Os dados foram limpos e unidos em um unico DataFrama, com MultiIndex para colunas e indexadas por um DateTimeIndex.

### Carregando os dados

In [ ]:
import pandas as pd

estacao_ssa_2003 = pd.read_csv('./datasets/estacao_Salvador_2003.CSV', header=8, sep=';', encoding='latin-1')
estacao_ssa_2013 = pd.read_csv('./datasets/estacao_Salvador_2013.CSV', header=8, sep=',', encoding='utf-8')
estacao_ssa_2023 = pd.read_csv('./datasets/estacao_Salvador_2023.CSV', header=8, sep=';', encoding='latin-1')
print()

### Tratamento de dados

1. Removendo coluna extra unnamed

In [ ]:
estacao_ssa_2003.drop(columns=['Unnamed: 19'], inplace=True)
estacao_ssa_2023.drop(columns=['Unnamed: 19'], inplace=True)

2. Unificando nomes de colunas

In [ ]:
# pegando lista de nomes de colunas de cada dataframe
colunas_2003 = estacao_ssa_2003.columns.values.tolist() 
colunas_2013 = estacao_ssa_2013.columns.values.tolist()
colunas_2023 = estacao_ssa_2023.columns.values.tolist()

# renomeando colunas diferentes para o nome da coluna do dataframe de 2003
for c_2003, c_2013, c_2023 in zip(colunas_2003, colunas_2013, colunas_2023):
    if c_2003 != c_2013 or c_2003 != c_2023:
        estacao_ssa_2013.rename(columns={c_2013: c_2003}, inplace=True)
        estacao_ssa_2023.rename(columns={c_2023: c_2003}, inplace=True)
        

# concatenando os dataframes
estacao_ssa = pd.concat([estacao_ssa_2003, estacao_ssa_2013, estacao_ssa_2023], ignore_index=True)

3. Criando DateTimeIndex

In [ ]:
# tratando coluna hora
mask_utc = estacao_ssa['HORA (UTC)'].str.contains('UTC')  # mascara para pegar apenas os dados com UTC

# trantando dados utc
dados_utc = estacao_ssa.loc[mask_utc, 'HORA (UTC)']
dados_utc = dados_utc.replace(' UTC', '', regex=True) # removendo UTC
dados_utc = dados_utc.str[0:2] + ':00' # pegando apenas a hora

# colocando os dados tratados no dataframe
estacao_ssa.loc[mask_utc, 'HORA (UTC)'] = dados_utc.values

In [ ]:
# tratando coluna data
estacao_ssa['DATA (YYYY-MM-DD)'] = estacao_ssa['DATA (YYYY-MM-DD)'].str.replace('/', '-')

In [ ]:
# criando DateTimeIndex a partir da coluna DATA (YYYY-MM-DD) e HORA (UTC)
estacao_ssa_index = pd.to_datetime(estacao_ssa['DATA (YYYY-MM-DD)'] + ' ' + estacao_ssa['HORA (UTC)'])

# removendo as colunas originais DATA (YYYY-MM-DD) e HORA (UTC)
estacao_ssa.drop(['DATA (YYYY-MM-DD)', 'HORA (UTC)'], axis=1, inplace=True)

# Define a coluna DATA HORA como índice
estacao_ssa.set_index(estacao_ssa_index, inplace=True)

4. Criando MultiIndex para colunas

In [ ]:
# agrupando parametros de medição
nivel_1 = ['precipitacao'] + ['pressao']*3 + ['radiacao'] + ['temperatura']*6 + ['umidade']*3 + ['velocidade do vento']*3
nivel_2 = colunas_2003[2:]

# definindo multiindex para as colunas
index_col = pd.MultiIndex.from_arrays([nivel_1, nivel_2], names=['medicao', 'tipo'])

estacao_ssa.columns = index_col

5. Removendo colunas desnecessárias

In [ ]:
estacao_ssa.drop(['pressao', 'radiacao', 'umidade', 'velocidade do vento'], axis=1, inplace=True)

6. Transformando colunas de string para float

In [ ]:
# substiduindo valores , por .
estacao_ssa.replace(',', '.', regex=True, inplace=True)

# convertendo para float
estacao_ssa = estacao_ssa.astype('float64')

7. Tratando valores nulos

In [ ]:
# tiranddo todos os possiveis valores nulos
estacao_ssa.replace(['-9999', -9999, -9999.0, '-9999.0'], value= None, inplace=True)

# preenchendo valores nulos
estacao_ssa.bfill(inplace=True)

### DataFrame final

In [ ]:
estacao_ssa.head()

In [ ]:
estacao_ssa.info()

In [ ]:
estacao_ssa.describe()

### Analise de dados

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

1. Subplot com evolução das temperaturas médias e acumulado de precipitações nos três anos.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

temperaturas_medias_anuais = estacao_ssa['temperatura'].groupby(estacao_ssa.index.year).mean()
acumulado_precipitacoes_anuais = estacao_ssa['precipitacao'].groupby(estacao_ssa.index.year).sum()

# Criando um subplot
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Definindo as cores para cada ano
cores_temperaturas = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'gray', 'pink', 'cyan', 'magenta', 'olive']
# Plotando a evolução das temperaturas médias
temperaturas_medias_anuais.plot.bar(ax=axs[0], color=cores_temperaturas, label='Temperatura Média')

# Configurações do primeiro subplot
axs[0].set_title('Evolução das Temperaturas Médias (2003-2023)')
axs[0].set_ylabel('Temperatura (°C)')

# Plotando o acumulado de precipitações
acumulado_precipitacoes_anuais.plot.bar(ax=axs[1], color='green', label='Acumulado de Precipitações')

# Configurações do segundo subplot
axs[1].set_title('Evolução do Acumulado de Precipitações (2003-2023)')
axs[1].set_ylabel('Precipitação (mm)')

# Adicionando legenda fora do gráfico
axs[0].legend(loc='upper left', bbox_to_anchor=(1, 1), title='Legenda 1')
axs[1].legend(loc='upper left', bbox_to_anchor=(1, 1), title='Legenda 2')

# Ajustando layout
plt.tight_layout()

2. Acumulado de precipitações de Junho nos três anos.

In [ ]:

dados_junho = estacao_ssa[('precipitacao', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)')][estacao_ssa.index.month == 6]

# Agrupando por ano e somar as precipitações de junho
acumulado_junho_por_ano = dados_junho.groupby([dados_junho.index.year, dados_junho.index.day]).sum().groupby(level=0).cumsum()

# Criando o gráfico de linha para o acumulado de precipitação
plt.figure(figsize=(8, 4))
cores = ['blue', 'green', 'red']  # Defina cores diferentes para cada ano

for i, (ano, dados) in enumerate(acumulado_junho_por_ano.groupby(acumulado_junho_por_ano.index.get_level_values(0))):
  plt.plot(dados.index.get_level_values(1), dados.values, color=cores[i], alpha=0.7, label=f'Ano {ano}', marker='o')  # Adicionando marcadores para cada ponto

plt.title('Acumulado de Precipitação em Junho')
plt.xlabel('Dia de Junho')
plt.ylabel('Acumulado de Precipitação (mm)')
plt.legend()
plt.show()


3. Temperatura media de cada um dos dias de Março nos três anos.

In [ ]:
#Temperatura do ar - Bulbo seco
dados_marco_bs = estacao_ssa[('temperatura', 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)')][estacao_ssa.index.month == 3]

temp_marco_bs = dados_marco_bs.groupby([dados_marco_bs.index.year, dados_marco_bs.index.day]).mean()


plt.figure(figsize=(10, 6))
cores = ['blue', 'green', 'red']

for i, (ano, dados) in enumerate(temp_marco_bs.groupby(temp_marco_bs.index.get_level_values(0))):
  plt.plot(dados.index.get_level_values(1), dados.values, color=cores[i], alpha=0.7, label=f'Ano {ano}', marker='o')  

plt.title('TEMPERATURA DO AR - BULBO SECO - MÊS MARÇO')
plt.xlabel('Dias de março')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.show()

In [ ]:
#Temperatura do ponto de orvalho
dados_marco_po = estacao_ssa[('temperatura', 'TEMPERATURA DO PONTO DE ORVALHO (°C)')][estacao_ssa.index.month == 3]

temp_marco_po = dados_marco_po.groupby([dados_marco_po.index.year, dados_marco_po.index.day]).mean()


plt.figure(figsize=(10, 6))
cores = ['blue', 'green', 'red']

for i, (ano, dados) in enumerate(temp_marco_po.groupby(temp_marco_po.index.get_level_values(0))):
  plt.plot(dados.index.get_level_values(1), dados.values, color=cores[i], alpha=0.7, label=f'Ano {ano}', marker='o')  

plt.title('TEMPERATURA PONTO DE ORVALHO - MÊS MARÇO')
plt.xlabel('Dias de março')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.show()

In [ ]:
#Temperatura na hora anterior
dados_marco_max = estacao_ssa[('temperatura', 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)')][estacao_ssa.index.month == 3]
dados_marco_min = estacao_ssa[('temperatura', 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)')][estacao_ssa.index.month == 3]

temp_marco_max = dados_marco_max.groupby([dados_marco_max.index.year, dados_marco_max.index.day]).mean()
temp_marco_min = dados_marco_min.groupby([dados_marco_min.index.year, dados_marco_min.index.day]).mean()

temp_marco_media = (temp_marco_max + temp_marco_min)/2
#temp_marco_media

plt.figure(figsize=(10, 6))
cores = ['blue', 'green', 'red']

for i, (ano, dados) in enumerate(temp_marco_media.groupby(temp_marco_media.index.get_level_values(0))):
  plt.plot(dados.index.get_level_values(1), dados.values, color=cores[i], alpha=0.7, label=f'Ano {ano}', marker='o')  

plt.title('TEMPERATURA MÉDIA NA HORA ANT. - MÊS MARÇO')
plt.xlabel('Dias de março')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.show()

In [ ]:
#Temperatura orvalho
dados_marco_omax = estacao_ssa[('temperatura', 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)')][estacao_ssa.index.month == 3]
dados_marco_omin = estacao_ssa[('temperatura', 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)')][estacao_ssa.index.month == 3]

temp_marco_omax = dados_marco_omax.groupby([dados_marco_omax.index.year, dados_marco_omax.index.day]).mean()
temp_marco_omin = dados_marco_omin.groupby([dados_marco_omin.index.year, dados_marco_omin.index.day]).mean()

temp_marco_omedia = (temp_marco_omax + temp_marco_omin)/2

plt.figure(figsize=(10, 6))
cores = ['blue', 'green', 'red']

for i, (ano, dados) in enumerate(temp_marco_omedia.groupby(temp_marco_omedia.index.get_level_values(0))):
  plt.plot(dados.index.get_level_values(1), dados.values, color=cores[i], alpha=0.7, label=f'Ano {ano}', marker='o')  

plt.title('TEMPERATURA ORVALHO MÉDIA NA HORA ANT. - MÊS MARÇO')
plt.xlabel('Dias de março')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.show()

4. Temperatura máxima e mínima de cada uma das estações nos tres anos. Sendo:
   - Verão (Janeiro, Fevereiro e Março)
   - Outono (Abril, Maio e Junho) 
   - Inverno (Julho, Agosto e Setembro)
   - Primavera (Outubro, Novembro e Dezembro).

In [ ]:
# criando coluna para estacao
dict_estacao = {1: 'verao', 2: 'verao', 3: 'verao', 
                4: 'outono', 5: 'outono', 6: 'outono', 
                7: 'inverno', 8: 'inverno', 9: 'inverno', 
                10: 'primavera', 11: 'primavera', 12: 'primavera'}

df_estacao = estacao_ssa['temperatura'].copy()
df_estacao['estacao'] = df_estacao.index.month.map(dict_estacao)
df_estacao.head()

In [ ]:
# temperaturas minimas
fig, ax = plt.subplots(2, 2, figsize = (13, 10), sharey=True)
nome_estacoes = ['verao', 'outono', 'inverno', 'primavera']
eixos = [(0, 0), (0, 1), (1, 0), (1, 1)]
anos = df_estacao.index.year.unique()

for i, estacao in zip(eixos, nome_estacoes):
    # pegando temperaturas médias por dia do ano de cada estação
    temp_min = df_estacao.query('estacao == @estacao')['TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)'].resample('D').mean()
    
    # plotando cada ano
    for ano in anos:
        ax[i].plot(temp_min[str(ano)].index.dayofyear, temp_min[str(ano)].values, label=f'Ano {ano}', )
        
        # configurando eixos
        ax[i].set_title(f'Temperaturas mínimas médias por dia do ano - {estacao}')
        ax[i].set_xlabel('Dia do ano')
        ax[i].set_ylabel('Temperatura (°C)')
        
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center')



In [ ]:
# temperaturas maximas
fig, ax = plt.subplots(2, 2, figsize = (13, 10), sharey=True)
nome_estacoes = ['verao', 'outono', 'inverno', 'primavera']
eixos = [(0, 0), (0, 1), (1, 0), (1, 1)]
anos = df_estacao.index.year.unique()

for i, estacao in zip(eixos, nome_estacoes):
    # pegando temperaturas médias por dia do ano de cada estação
    temp_max = df_estacao.query('estacao == @estacao')['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'].resample('D').mean()
    
    # plotando cada ano
    for ano in anos:
        ax[i].plot(temp_max[str(ano)].index.dayofyear, temp_max[str(ano)].values, label=f'Ano {ano}', )
        
        # configurando eixos
        ax[i].set_title(f'Temperaturas máximas médias por dia do ano - {estacao}')
        ax[i].set_xlabel('Dia do ano')
        ax[i].set_ylabel('Temperatura (°C)')
        
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center')

5. Comportamento de temperatura media do dia 28 de Setembro em cada um dos intervalos do dia dos três anos. Sendo:
   - Madrugada (00:00 a 06:00), 
   - Manha (06:00 a 12:00)
   - Tarde (12:00 a 18:00)
   - Noite (18:00 a 00:00)

In [ ]:
dados_dia_28_set = estacao_ssa[(estacao_ssa.index.month == 9) & (estacao_ssa.index.day == 28)]

In [ ]:
# Média Madrugada

media_madrugada = dados_dia_28_set[(dados_dia_28_set.index.hour >= 0) & (dados_dia_28_set.index.hour < 6)]
media_madrugada_por_ano = media_madrugada.groupby(media_madrugada.index.year).mean()

In [ ]:
# Média Manhã

media_manha = dados_dia_28_set[(dados_dia_28_set.index.hour >= 6) & (dados_dia_28_set.index.hour < 12)]
media_manha_por_ano = media_manha.groupby(media_manha.index.year).mean()

In [ ]:
# Média Tarde

media_tarde = dados_dia_28_set[(dados_dia_28_set.index.hour >= 12) & (dados_dia_28_set.index.hour < 18)]
media_tarde_por_ano = media_tarde.groupby(media_tarde.index.year).mean()

In [ ]:
# Média Noite

media_noite = dados_dia_28_set[(dados_dia_28_set.index.hour >= 18) & (dados_dia_28_set.index.hour < 24)]
media_noite_por_ano = media_noite.groupby(media_noite.index.year).mean()

In [ ]:
plt.figure(figsize=(10, 6))

plt.ylabel('Temperatura (°C)')
plt.title('Média de Temperatura por Ano')

for ano in media_madrugada_por_ano.index:
    plt.plot(['Madrugada', 'Manhã', 'Tarde', 'Noite'], [media_madrugada_por_ano['temperatura']['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'][ano],
                                                       media_manha_por_ano['temperatura']['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'][ano],
                                                       media_tarde_por_ano['temperatura']['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'][ano],
                                                       media_noite_por_ano['temperatura']['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'][ano]], label=ano, marker='o')

plt.legend()